In [6]:
from numpy.core.numeric import normalize_axis_tuple
import pandas as pd
from pandas.core.arrays import categorical#install panda first
!pip install sdv gensim unidecode contractions
import sdv #install sdv first (see https://sdv.dev/SDV/user_guides/)
from sdv.metrics.tabular import NumericalLR
from sdv.demo import load_tabular_demo
from sdv.tabular import GaussianCopula
from sdv.metrics.tabular import NumericalLR
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk, gensim, string, re
import unidecode, contractions, string, re
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')


from sdv.metrics.tabular import CategoricalCAP
from sdv.metrics.tabular import CategoricalZeroCAP
from sdv.evaluation import evaluate
from gensim.models import Word2Vec, Phrases
from gensim.test.utils import common_texts

from google.colab import drive
drive.mount('/content/drive')

stopwords = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
porterstemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path = "drive/My Drive/DataSifter/Data/mimic3.csv"
real_data = pd.read_csv(path)
data_copy = real_data.copy()[:50]

feature = 'marriage'
print(common_texts)

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]


In [3]:
# diagnoses_df = real_data['TEXT']
diagnoses_df = data_copy['TEXT']

# remove stop words 
diagnoses_df = diagnoses_df.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stopwords))
# texts 
diagnoses_list = list(diagnoses_df)
diagnoses_list[3]

'name     known lastname             known firstname     unit no     numeric identifier          admission date                  discharge date                  date birth                  sex  service  medicine allergies  sulfa  sulfonamide antibiotics    vasotec   ranitidine attending    first name   lf          addendum  changed aspirin    mg atorvastatin crestor    mg per pt request  medications admission  preadmission medications listed correct complete  information obtained webomr     losartan potassium     mg po daily    labetalol     mg po bid    fish oil  omega         mg po daily    metformin xr  glucophage xr      mg po daily crush    fluticasone propionate nasal   spry nu daily    amlodipine    mg po daily    atorvastatin    mg po daily    furosemide    mg po daily    sildenafil  nf      mg oral prn ed     omeprazole    mg po daily     potassium chloride    meq po tid duration     hours hold k       aspirin     mg po daily     cetirizine  nf     mg oral daily     citalopram

Preprocessing starts

In [4]:
def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = contractions.fix(text)
    return text
  
def decontracted(phrase):
  """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""

  # specific
  phrase = re.sub(r"won\'t", "will not", phrase)
  phrase = re.sub(r"can\'t", "can not", phrase)
  phrase = re.sub(r"won\’t", "will not", phrase)
  phrase = re.sub(r"can\’t", "can not", phrase)

  # general
  phrase = re.sub(r"n\'t", " not", phrase)
  phrase = re.sub(r"\'re", " are", phrase)
  phrase = re.sub(r"\'s", " is", phrase)
  phrase = re.sub(r"\'d", " would", phrase)
  phrase = re.sub(r"\'ll", " will", phrase)
  phrase = re.sub(r"\'t", " not", phrase)
  phrase = re.sub(r"\'ve", " have", phrase)
  phrase = re.sub(r"\'m", " am", phrase)

  phrase = re.sub(r"n\’t", " not", phrase)
  phrase = re.sub(r"\’re", " are", phrase)
  phrase = re.sub(r"\’s", " is", phrase)
  phrase = re.sub(r"\’d", " would", phrase)
  phrase = re.sub(r"\’ll", " will", phrase)
  phrase = re.sub(r"\’t", " not", phrase)
  phrase = re.sub(r"\’ve", " have", phrase)
  phrase = re.sub(r"\’m", " am", phrase)

  return phrase

def preprocessing(texts):
  tokenized_texts = []
  for i, text in enumerate(texts):
    texts[i] = decontracted(texts[i])
    text_words_list = word_tokenize(text)
    text_words_list = [expand_contractions(word) for word in text_words_list]
    text_words_list = [porterstemmer.stem(word) for word in text_words_list]
    text_words_list = [lemmatizer.lemmatize(word) for word in text_words_list]
    for j, word in enumerate(text_words_list):
      # set all words to lower case
      text_words_list[j] = text_words_list[j].lower() 
    texts[i] = ' '.join(text_words_list)
    tokenized_texts.append(text_words_list)
  return texts, tokenized_texts
# tokenized_diagonese is the input for w2v
diagnoses_list, tokenized_diagoses = preprocessing(diagnoses_list)
print(len(diagnoses_list))
print(tokenized_diagoses)

50
[['admiss', 'date', 'discharg', 'date', 'date', 'birth', 'sex', 'servic', 'neurosurgeri', 'allergi', 'codein', 'streptokinas', 'iodin', 'bee', 'pollen', 'narcan', 'attend', 'first', 'name', 'lf', 'chief', 'complaint', 'worst', 'headach', 'life', 'major', 'surgic', 'invas', 'procedur', 'cerebr', 'angiogram', 'right', 'evd', 'placement', 'right', 'evd', 'cathet', 'replac', 'ivc', 'filter', 'placement', 'ventriculoperiton', 'shunt', 'histori', 'present', 'ill', 'year', 'old', 'man', 'aspirin', 'coumadin', 'atrial', 'fibril', 'cva', 'medflight', 'hospit', 'today', 'follow', 'worst', 'headach', 'life', 'hour', 'ago', 'inr', 'head', 'ct', 'consist', 'extens', 'subarachnoid', 'hemorhag', 'patient', 'given', 'vitamin', 'k', 'mg', 'factor', 'revers', 'inr', 'upon', 'arriv', 'patient', 'daughter', 'state', 'past', 'day', 'develop', 'left', 'facial', 'droop', 'slur', 'speech', 'droopi', 'left', 'eye', 'weak', 'bilater', 'upper', 'extrem', 'sinc', 'care', 'accid', 'approxim', 'week', 'ago', 'ba

In [17]:
bigram_transformer = Phrases(tokenized_diagoses)
basic_model = Word2Vec(sentences=diagnoses_list, size=100, window=5, min_count=1, workers=2, hs=1, negative=0)
bigram_model = Word2Vec(bigram_transformer[tokenized_diagoses], min_count=1)
basic_model.train(tokenized_diagoses[:len(tokenized_diagoses)*9//10], total_examples=len(tokenized_diagoses), epochs=1)
basic_model.score(sentences=diagnoses_df[46:])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


array([-35099.855, -35014.707, -35271.344, -35014.32 ], dtype=float32)

Step 4

In [ ]:
# For step 4 - 6: Dummy here
def getWords(feature):
    return ['always', 'he', 'she', 'i']

In [ ]:
feature_list = getWords(feature)
feature_vector_list = []

Step 5

In [ ]:
# Bert here

Step 6

In [ ]:
def recast(feature, data):
    df = data.copy()
    texts_list = list(df['TEXT'])
    key_words = getWords(feature)
    for key_word in key_words:
        df[key_word] = [0] * len(df)
    for i, text in enumerate(texts_list):
        for j, key_word in enumerate(key_words):
            if containKeyWords(key_word, text):
                df.iloc[i, j+9] = 1
    return df

In [ ]:
def containKeyWords(key_word, text):
    key_word = ps.stem(key_word)
    text_tokens = word_tokenize(text)
    
    text_tokens = [token.lower() for token in text_tokens]
    text_tokens = [token for token in text_tokens if token not in string.punctuation]
    text_tokens = [porterstemmer.stem(token) for token in text_tokens]
    return key_word in text_tokens

In [ ]:
print(recast(feature, data_copy))

    SUBJECT_ID  HADM_ID  EXPIRE_FLAG GENDER  ... always  he she  i
0        58242   139697            0      M  ...      1   0   1  0
1        99469   179324            1      M  ...      1   0   0  0
2        62150   164465            0      M  ...      1   0   1  0
3        92137   103726            0      M  ...      0   0   0  0
4        81583   133495            1      F  ...      1   1   0  0
5        71054   106938            1      M  ...      1   0   0  0
6          249   149546            0      F  ...      0   1   0  0
7        70097   137788            0      M  ...      1   0   0  0
8        73055   137944            0      F  ...      0   1   0  0
9        80764   166769            0      M  ...      1   0   0  0
10       88632   144902            1      M  ...      1   0   1  0
11       91529   182338            0      F  ...      0   1   1  0
12       70188   195501            0      M  ...      1   0   0  0
13         249   158975            0      F  ...      0   1   

Step 7

In [ ]:
def create_table():
    table = pd.DataFrame(columns=['metric', 'name', 'normalized_score_small', 'normalized_score_med', 'normalized_score_large'])
    eval = [evaluate(synthetic_data[i], real_data,  aggregate=False) for i in range(3)] 
    eval_small = eval[0]
    eval_med = eval[1]
    eval_large = eval[2]
    metrics_list = list(eval_small.metric)
    print(metrics_list + ['privacy'])
    table.metric = metrics_list + ['privacy']
    
    table.name = list(eval_small.name) + ['categorical']  

    privacy_score = [CategoricalCAP.compute(
        real_data.fillna(0),
        synthetic_data[i].fillna(0),
        key_fields=['TEXT'],
        sensitive_fields=['GENDER']
        ) for i in range(len(synthetic_data))]
    table.normalized_score_small = list(eval_small.normalized_score) + [privacy_score[0]]
    table.normalized_score_med = list(eval_med.normalized_score) + [privacy_score[1]]
    table.normalized_score_large = list(eval_large.normalized_score) + [privacy_score[2]]
    return table

In [ ]:
model = GaussianCopula()
real_data_0 = real_data.iloc[0:50]
# real_data_1 = real_data.iloc[51:100]
# real_data_2 = real_data.iloc[101:150]
real_data = real_data_0
# real_data
model.fit(real_data)
synthetic_data_0 = model.sample()
# model.fit(real_data_1)
synthetic_data_1 = model.sample()
# model.fit(real_data_2)
synthetic_data_2 = model.sample()
synthetic_data = [synthetic_data_0, synthetic_data_1, synthetic_data_2]
# metrics_list = ['CSTest', 'KSTest']
table = create_table()
 


/usr/local/lib/python3.7/dist-packages/scipy/stats/_continuous_distns.py:5320: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/usr/local/lib/python3.7/dist-packages/scipy/stats/_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/usr/local/lib/python3.7/dist-packages/scipy/stats/_continuous_distns.py:639: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/usr/local/lib/python3.7/dist-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/sdmetrics/column_pairs/statistical/kl_divergence.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

['BNLogLikelihood', 'LogisticDetection', 'SVCDetection', 'BinaryDecisionTreeClassifier', 'BinaryAdaBoostClassifier', 'BinaryLogisticRegression', 'BinaryMLPClassifier', 'MulticlassDecisionTreeClassifier', 'MulticlassMLPClassifier', 'LinearRegression', 'MLPRegressor', 'GMLogLikelihood', 'CSTest', 'KSTest', 'KSTestExtended', 'CategoricalCAP', 'CategoricalZeroCAP', 'CategoricalGeneralizedCAP', 'CategoricalNB', 'CategoricalKNN', 'CategoricalRF', 'CategoricalSVM', 'CategoricalEnsemble', 'NumericalLR', 'NumericalMLP', 'NumericalSVR', 'NumericalRadiusNearestNeighbor', 'ContinuousKLDivergence', 'DiscreteKLDivergence', 'privacy']
